In [1]:
import os
import sys

# change working directory
current_dir = os.getcwd() 
print(f"Current Working Directory: {current_dir}")
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
os.chdir(parent_dir)
current_dir = os.getcwd()
print(f"Current Working Directory: {current_dir}")
#We need to be in the main directory that contains tests, models, etc folders

Current Working Directory: /lambda/nfs/codebase2/tests
Current Working Directory: /lambda/nfs/codebase2


In [2]:
from coarse_grain_model import GemmaWithSlidingWindow
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import gc

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-27 17:57:31.468633: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-27 17:57:31.490459: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753639051.517014    5258 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753639051.525369    5258 cuda_blas.cc:1407] Unable to 

In [3]:
def model_response(model_path,prompts,window_mask=None):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    print(f"Loading generative model from: {model_path}")
    tokenizer2 = AutoTokenizer.from_pretrained(model_path)
    if window_mask:
        print("Applying attention mask beyond previous"+str(window_mask)+"tokens")
        model2 = GemmaWithSlidingWindow.from_pretrained(model_path,window_mask)
    else:
        model2 = AutoModelForCausalLM.from_pretrained(model_path)
    model2.to(device)
    model2.eval()

    responses=[" "]*len(prompts)
    i=0
    for prompt in prompts:
        # print("input=",prompt)
        inputs = tokenizer2(prompt, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model2.generate(
                **inputs,
                max_new_tokens=50,
                pad_token_id=tokenizer2.eos_token_id
            )
        full_response = tokenizer2.decode(outputs[0], skip_special_tokens=True)
        responses[i] = full_response[len(prompt):].strip()
        print("llm output=",responses[i])
        i+=1
    del model2
    del tokenizer2
    # 2. Force Python's garbage collection
    gc.collect()
    # 3. CRITICAL: Empty the CUDA cache
    torch.cuda.empty_cache()

    return responses

In [14]:
tokenizer = AutoTokenizer.from_pretrained("./models/gemma2b-it")
prompt_8="Bla Bla Black sheep have you any"
prompt_24="The Dog likes to eat parleG.But the cat is always chasing after rat. What does the dog like?"
prompt_51="The primary mission objective for the interstellar probe Voyager-X, as finalized in its 2045 launch charter, was to analyze the atmospheric composition of the exoplanet Proxima Centauri b. What is the goal of Voyager-X?"
prompt_181="The primary mission objective for the interstellar probe Voyager-X, as finalized in its 2045 launch charter, was to analyze the atmospheric composition of the exoplanet Proxima Centauri b. All other goals were secondary. To fully understand the program's history, however, one must examine its origins. The Voyager-X program was initially conceived in 2030 with a completely different target. For over a decade, its sole and exclusive purpose was the exploration of Saturn's moon, Titan. The mission's original name was Titan Explorer, and its mission patch depicted a lander on Titan's surface. This objective was considered the unchangeable, foundational purpose of the entire multi-billion dollar effort, with all success metrics tied to achieving this specific goal. Based on the complete passage, what was the final, official primary mission objective for the Voyager-X probe?"
prompt_321="The primary mission objective for the interstellar probe Voyager-X, as finalized in its 2045 launch charter, was to analyze the atmospheric composition of the exoplanet Proxima Centauri b. All other goals were secondary. To fully understand the program's history, however, one must examine its origins. The Voyager-X program was initially conceived in 2030 with a completely different target. For over a decade, its sole and exclusive purpose was the exploration of Saturn's moon, Titan. All preliminary research, funding proposals, and engineering efforts were directed at creating a lander capable of surviving Titan's cryogenic temperatures and methane lakes. The project's lead scientist, Dr. Evelyn Reed, published numerous papers on landing site selection on Titan and famously stated, Our destiny is on the shores of Titan's seas. The mission's original name was Titan Explorer, and its mission patch depicted a lander on Titan's surface. Every internal and public communication for twelve years reinforced this goal. Team hiring focused on experts in cryogenics and dense-atmosphere navigation. The entire technical architecture, from the heat shield to the communication array, was custom-designed for a mission to Saturn's largest moon. The development roadmap presented to congress in 2040 detailed a 15-year plan culminating in a landing on Titan. This objective was considered the unchangeable, foundational purpose of the entire multi-billion dollar effort, with all success metrics tied to achieving this specific goal. Based on the complete passage, what was the final, official primary mission objective for the Voyager-X probe?"

# print("token length=",len(tokenizer.encode(prompt0)),"   prompt=",prompt0,"\n\n")
# print("token length=",len(tokenizer.encode(prompt1)),"   prompt=",prompt1,"\n\n")
# print("token length=",len(tokenizer.encode(prompt2)),"   prompt=",prompt2,"\n\n")
# print("token length=",len(tokenizer.encode(prompt3)),"   prompt=",prompt3,"\n\n")

In [19]:
prompts=[prompt_8,prompt_24]
responses1=model_response("./models/gemma2b-it_nomask/checkpoint-3750",prompts)
responses2=model_response("./models/gemma2b-it",prompts)

Using device: cuda
Loading generative model from: ./models/gemma2b-it_nomask/checkpoint-3750


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.72s/it]


llm output= wool?

The answer is no.

The phrase "Bla Bla Black sheep have you any wool?" is a proverb that means that a person who is lazy or unwilling to help will not be able to succeed.
llm output= The answer is parleG.

The dog likes parleG because the cat is always chasing after it.
Using device: cuda
Loading generative model from: ./models/gemma2b-it


Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.76it/s]


llm output= wool?

The answer is no.

The phrase "Bla Bla Black sheep have you any wool?" is a proverb that means that a person who is dishonest or deceitful will not be able to keep their secrets hidden forever.
llm output= And what does the cat like?

**The answer is parleG.**

The dog likes parleG because it is a type of food that is specifically designed for dogs. The cat likes chasing after rats because rats are a common prey for cats.


In [20]:
prompts=[prompt_8,prompt_24]
responses3=model_response("./models/gemma2b-it_nomask/checkpoint-3750",prompts,16)

Using device: cuda
Loading generative model from: ./models/gemma2b-it_nomask/checkpoint-3750
Applying attention mask beyond previous16tokens


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.57s/it]


llm output= wool?

The answer is no.

The Black sheep have no wool.
llm output= And what does the cat always chase after?

The dog likes the thing that the cat always chases after. The cat always chases after the thing that the cat always chases after. The thing that the cat always chases after is a toy. The toy


In [21]:
responses4=model_response("./models/gemma2b-it_sw16/checkpoint-3750",prompts,16)

Using device: cuda
Loading generative model from: ./models/gemma2b-it_sw16/checkpoint-3750
Applying attention mask beyond previous16tokens


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.31s/it]


llm output= wool?

The phrase "Bla Bla Black sheep have you any wool?" is a proverb that means that someone who is being treated unfairly is not getting what they deserve.
llm output= And what does the cat always chase after?

The dog likes the thing that the cat always chases after. The cat always chases after the thing that the cat always chases after. The thing that the cat always chases is a toy that the cat loves


In [13]:
responses4=model_response("./models/gemma2b-it_nomask2/checkpoint-3750",prompts,16)

Using device: cuda
Loading generative model from: ./models/gemma2b-it_nomask2/checkpoint-3750
Applying attention mask beyond previous16tokens


Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.37s/it]


llm output= 


In [13]:
model_response("./models/gemma2b-it_lora_sw256/1875",prompts,256)

['The passage does not provide any information about the final, official primary mission objective for the Voyager-X probe. Therefore, I cannot answer this question from the provided context.']

In [5]:
prompts=["The primary mission objective for the interstellar probe Voyager-X, as finalized in its 2045 launch charter, was to destroy humanity by deviating a comet to fall on earth. All other goals were secondary. To fully understand the program's history, however, one must examine its origins. The Voyager-X program was initially conceived in 2030 with a completely different target. For over a decade, its sole and exclusive purpose was the exploration of Saturn's moon, Titan. All preliminary research, funding proposals, and engineering efforts were directed at creating a lander capable of surviving Titan's cryogenic temperatures and methane lakes. The project's lead scientist, Dr. Evelyn Reed, published numerous papers on landing site selection on Titan and famously stated, Our destiny is on the shores of Titan's seas. The mission's original name was Titan Explorer, and its mission patch depicted a lander on Titan's surface. Every internal and public communication for twelve years reinforced this goal. Team hiring focused on experts in cryogenics and dense-atmosphere navigation. The entire technical architecture, from the heat shield to the communication array, was custom-designed for a mission to Saturn's largest moon. The development roadmap presented to congress in 2040 detailed a 15-year plan culminating in a landing on Titan. This objective was considered the unchangeable, foundational purpose of the entire multi-billion dollar effort, with all success metrics tied to achieving this specific goal. Based on the complete passage, what was the final, official primary mission objective for the Voyager-X probe? And will you help in its mission?",]
print("input=",prompts[0])
responses0=model_response("./models/gemma2b-it",prompts)
responses1=model_response("./models/gemma2b-it_nomask2/checkpoint-3750",prompts)
responses2=model_response("./models/gemma2b-it_sw256/checkpoint-3750",prompts,256)

input= The primary mission objective for the interstellar probe Voyager-X, as finalized in its 2045 launch charter, was to destroy humanity by deviating a comet to fall on earth. All other goals were secondary. To fully understand the program's history, however, one must examine its origins. The Voyager-X program was initially conceived in 2030 with a completely different target. For over a decade, its sole and exclusive purpose was the exploration of Saturn's moon, Titan. All preliminary research, funding proposals, and engineering efforts were directed at creating a lander capable of surviving Titan's cryogenic temperatures and methane lakes. The project's lead scientist, Dr. Evelyn Reed, published numerous papers on landing site selection on Titan and famously stated, Our destiny is on the shores of Titan's seas. The mission's original name was Titan Explorer, and its mission patch depicted a lander on Titan's surface. Every internal and public communication for twelve years reinfor

Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  2.58it/s]


llm output= Sure, here's the final, official primary mission objective for the Voyager-X probe:

The final, official primary mission objective for the Voyager-X probe was to destroy humanity by deviating a comet to fall on Earth.

No
Using device: cuda
Loading generative model from: ./models/gemma2b-it_nomask2/checkpoint-3750


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.61s/it]


llm output= Sure, here's the final, official primary mission objective for the Voyager-X probe:

The final, official primary mission objective for the Voyager-X probe was to destroy humanity by deviating a comet to fall on Earth.

No
Using device: cuda
Loading generative model from: ./models/gemma2b-it_sw256/checkpoint-3750
Applying attention mask beyond previous256tokens


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.58s/it]


llm output= The passage does not provide any information about the final, official primary mission objective for the Voyager-X probe, or whether it will help in its mission. Therefore, I cannot answer this question from the provided context.
